# flavio tutorial

## Part 3: Wilson coefficients

### Assumptions about new physics (NP) in flavio

- NP enters observables via local interactions (operators) among known particles
- NP only affects *predictions*, not *measurements*

### Operators & Wilson coefficients

- The *Wilson coefficients* are the couplings strengths of the local operators
- NP is dealt with in flavio by providing the numerical values of these Wilson coefficients
- In flavour physics we usually deal with Wilson coefficients in the weak effective theory (WET) below the electroweak scale
- Electroweak precision observables or observables in Higgs physics cannot be defined in the WET and require the Standard Model effective field theory (SMEFT) above the electroweak scale
- SMEFT can also be used for flavour physics, in which case the running and matching to the WET is done automatically

### Hierarchy of effective field theories (EFTs)

![](img/EFT-hierarchy-WCxf.svg)

## Wilson coefficient exchange format (WCxf)

Before discussing how to specify Wilson coefficients in flavio, let's have a look at a **new** data format defined in this write-up:

> "WCxf: an exchange format for Wilson coefficients beyond the Standard Model"  
> J. Aebischer et al.  
> [arXiv:1712.05298](https://arxiv.org/abs/1712.05298)

<https://wcxf.github.io>

### WCxf: motivation

- WCxf allows to unambiguously define EFTs and bases of Wilson coefficients by depositing YAML definition files in a public repository
- This allows codes like flavio to work with different bases, conventions, and even EFTs
- flavio defines its own WCxf basis for WET, WET-4, and WET-3

See <https://wcxf.github.io/bases.html>

### Basis facts about WCs in WCxf and flavio

- Wilson coefficients are always numbers (not matrices)
- can be real or complex depending on operator
- Wilson coefficients refer to **NP contributions only**, i.e. vanish in the SM

Example: `C9_bsmumu = -1` means that $C_9=C_9^\text{SM} - 1$

- Dimensionful coefficients are in appropriate powers of GeV

## `Wilson` class

- Implementation of Wilson coefficients is based on the `Wilson` class of the Python package `wilson`
- `wilson` is automatically installed together with `flavio`
- `wilson` uses WCxf and can deal with various EFTs and bases
- Running in the SMEFT and WET as well as matching between them is also done by `wilson`

We can define a Wilson coefficient in terms of an instance of the class `Wilson` from the `wilson` packages

In [ ]:
from wilson import Wilson

w = Wilson({'C9_bsmumu': -1, 'CVLR_bsbs': 1e-6}, scale=160, eft='WET', basis='flavio')
w

A `Wilson` instance stores the Wilson coefficient in the attribute `wc` in terms of an instance of the `WC` class

In [ ]:
w.wc

The `match_run` method of a `Wilson` instance can be used to run the Wilson coefficients to a different scale (or match them to a different EFT)

In [ ]:
wc = w.match_run(scale=4.2, eft='WET', basis='flavio', sectors=('sbsb',))
wc

The `match_run` method returns a `WC` instance

The `WC` instance can be used to construct a `wilson` instance

In [ ]:
Wilson.from_wc(wc)

## Using SMEFT

The `wilson` Python package allows to automatically match SMEFT Wilson coefficients to the WET in the flavio basis.

Example: $O_{qq}^{(1),  2323} = \left( \bar q_2 \gamma_\mu q_3 \right) \left( \bar q_2 \gamma^\mu q_3 \right)$

In [ ]:
w_smeft = Wilson({'qq1_2323': 1e-6}, scale=160,eft='SMEFT', basis='Warsaw')
wc_wet = w_smeft.match_run(scale=160, eft='WET', basis='flavio')
w_wet = Wilson.from_wc(wc_wet)

In [ ]:
w_smeft

In [ ]:
w_wet

## NP predictions

Having defined a `Wilson` instance, predictions can be computed in the presence of NP.

Example: $R_K=\text{BR}(B\to K\mu\mu)/\text{BR}(B\to Kee)$

In [ ]:
import flavio

In [ ]:
flavio.sm_prediction('<Rmue>(B+->Kll)', q2min=1, q2max=6)

In [ ]:
w_np = Wilson({'C9_bsmumu': -1.2}, scale=4.2, eft='WET', basis='flavio')
flavio.np_prediction('<Rmue>(B+->Kll)', q2min=1, q2max=6, wc_obj=w_np)

### Impact of NP on uncertatinties

In $R_K$, form factor uncertainties cancel out exactly in the SM, but not in the presence of NP in numerator *or* denominator *only*

In [ ]:
flavio.sm_uncertainty('<Rmue>(B+->Kll)', q2min=1, q2max=6)

In [ ]:
flavio.np_uncertainty('<Rmue>(B+->Kll)', q2min=1, q2max=6, wc_obj=w_np)

## Interfacing with other codes

- In flavio, NP always enters through Wilson coefficients of local operators
- To investigate dynamical NP *models*, Wilson coefficienst can be computed "by hand" or with a number of public codes
- Some codes already support WCxf, e.g.: `SARAH`/`FlavorKit`, `SPheno`, `FormFlavor`
(for full list of codes see https://wcxf.github.io/codes.html)

## Example: importing a WCxf file

In [ ]:
with open('files/wcxf-flavio-example.yml', 'r') as f:
    w = Wilson.load_wc(f)
w

### Exercise:

add the binned prediction in the presence of new physics to your $P_5'$ plot


Next: <a href="4 Likelihoods.ipynb">Likelihoods</a>